## Plan

- Go through nonfungible, find latest kitty that has same id/USD and date!
- create a df, and fill it with stuff from cryptokittie api
- append this df ontop of `train_dense.csv`


In [52]:
import pandas as pd
import requests
import json
import warnings
import os
import codecs
warnings.filterwarnings('ignore')

pd.set_option('max_columns', 30)

    
# data = pd.read_csv('train_dense.csv')
# data.head()

In [79]:
pd.read_csv('train_complete.csv')

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,2019-07-07T06:37:59.000Z,1561681,1.10,5.0,eclipse,False,False,NaN,False,NaN,NaN,NaN,NaN,selkirk,splat,wiley,ducky,fangtastic,greymatter,royalpurple,frosting,3.0,True,False,0.0,False,0.0,False
1,2019-07-07T06:32:17.000Z,1553924,0.32,8.0,doridnudibranch,False,False,NaN,False,NaN,NaN,NaN,NaN,spock,ragdoll,baddate,salty,rollercoaster,ducky,greymatter,cerulian,9.0,True,False,0.0,False,0.0,False
2,2019-07-07T06:31:26.000Z,1555773,0.32,7.0,coralsunrise,False,False,NaN,False,NaN,NaN,NaN,NaN,thicccbrowz,moue,tiger,salty,highlander,ducky,salmon,swampgreen,7.0,True,False,0.0,False,0.0,False
3,2019-07-07T06:29:29.000Z,1554209,0.34,6.0,sapphire,False,False,NaN,False,NaN,NaN,NaN,NaN,totesbasic,savannah,moue,baddate,salty,ducky,salmon,coffee,9.0,True,False,0.0,False,0.0,False
4,2019-07-07T06:26:32.000Z,1552162,0.33,8.0,sapphire,False,False,NaN,False,NaN,NaN,NaN,NaN,thicccbrowz,grim,salty,koladiviya,rorschach,ducky,cottoncandy,swampgreen,12.0,False,False,0.0,False,0.0,False
5,2019-07-07T06:25:51.000Z,1151860,26.89,11.0,kaleidoscope,False,False,NaN,False,NaN,NaN,NaN,NaN,soserious,kaleidoscope,wiley,rascal,koladiviya,shadowgrey,royalpurple,flamingo,5.0,True,False,0.0,False,0.0,False
6,2019-07-07T06:22:56.000Z,1639876,0.35,9.0,parakeet,False,False,NaN,False,NaN,NaN,NaN,NaN,saycheese,ragdoll,spangled,wonky,salty,dragontail,bananacream,swampgreen,4.0,True,False,0.0,False,0.0,False
7,2019-07-07T06:17:30.000Z,1639762,0.36,7.0,strawberry,False,False,NaN,False,NaN,NaN,NaN,NaN,happygokitty,splat,wonky,savannah,salty,salmon,rosequartz,cobalt,3.0,True,False,0.0,False,0.0,False
8,2019-07-07T06:14:19.000Z,1554853,1.94,14.0,sapphire,False,False,NaN,False,NaN,NaN,NaN,NaN,himalayan,otaku,whixtensions,amur,salty,ducky,cottoncandy,coffee,9.0,True,False,1.0,False,0.0,False
9,2019-07-07T06:10:08.000Z,1639788,0.37,7.0,olive,False,False,NaN,False,NaN,NaN,NaN,NaN,sphynx,olive,spock,pouty,salty,swarley,ducky,mauveover,3.0,True,False,0.0,False,0.0,False


## FUNCTION TO PULL LATEST FROM NONFUNGIBLE

In [31]:
def update_train_dense(data):
    # finding latest data that I have
    latest_time = data['time'][0]
    latest_ID = data['ID'][0]
    latest_USD = data['USD'][0]

    # defining dict to store new data
    updated = {'results':[]}

    # for loop with arbitrarily large number 
    for i in range(500):
        data_points = 80
        start = data_points*i # Starting point to collect data
        URL = "https://nonfungible.com/api/v1/market/cryptokitties/history/?start={}&length={}&columns[0][name]=blockTimestamp&order[0][dir]=desc".format(start, data_points)

        # using requests to get json data, converting into python dict
        r = requests.get(url = URL)
        data = r.json()

        for k in range(data_points):
            if data['data'][k]['saleType'] != 'mint':
                ID = data['data'][k]['assetId']
                USD = data['data'][k]['usdPrice']
                time = data['data'][k]['blockTimestamp']
                print(time, ID, USD, end = "\r")

                if (str(time) == latest_time) and (int(ID) == latest_ID) and (float(USD) == latest_USD):
                    print("FOUND LATEST POINT!")
                    return updated


                updated['results'].append([time, ID, USD])
    #         print(time, start)

    
    

In [34]:
results = update_train_dense(data)

2019-07-07T06:37:59.000Z 1561681 1.1
2019-07-07T06:32:17.000Z 1553924 0.32
2019-07-07T06:31:26.000Z 1555773 0.32
2019-07-07T06:29:29.000Z 1554209 0.34
2019-07-07T06:26:32.000Z 1552162 0.33
2019-07-07T06:25:51.000Z 1151860 26.89
2019-07-07T06:22:56.000Z 1639876 0.35
2019-07-07T06:17:30.000Z 1639762 0.36
2019-07-07T06:14:19.000Z 1554853 1.94
2019-07-07T06:10:08.000Z 1639788 0.37
2019-07-07T05:59:15.000Z 1549581 0.37
2019-07-07T05:58:13.000Z 1555728 0.36
2019-07-07T05:51:18.000Z 525619 2.89
2019-07-07T05:26:09.000Z 1010032 26.39
2019-07-07T05:26:09.000Z 1624399 1.95
2019-07-07T05:11:42.000Z 1566391 3
2019-07-07T05:09:24.000Z 589486 50.46
2019-07-07T05:08:17.000Z 1455614 3.36
2019-07-07T05:02:56.000Z 1012168 2.9
2019-07-07T04:57:03.000Z 1043670 35.7
2019-07-07T04:31:54.000Z 1307396 3.76
2019-07-07T04:29:45.000Z 1630643 5.54
2019-07-07T04:26:19.000Z 857510 5.43
2019-07-07T04:25:24.000Z 1153115 1.28
2019-07-07T04:25:12.000Z 1306036 1.42
2019-07-07T04:21:50.000Z 1549907 1.02
2019-07-07T04:15:

2019-07-06T17:21:44.000Z 1561684 2.8
2019-07-06T17:21:34.000Z 1561656 2.8
2019-07-06T17:19:05.000Z 1565199 1.75
2019-07-06T17:18:56.000Z 1562124 0.59
2019-07-06T17:17:48.000Z 1565885 2.36
2019-07-06T17:17:48.000Z 1562148 1.47
2019-07-06T17:16:32.000Z 1590267 0.95
2019-07-06T17:16:08.000Z 1629360 4.38
2019-07-06T17:15:33.000Z 1323348 2.65
2019-07-06T17:15:33.000Z 1562837 2.65
2019-07-06T17:15:33.000Z 1631042 4.8
2019-07-06T17:14:12.000Z 1562963 1.11
2019-07-06T17:13:28.000Z 1553736 0.59
2019-07-06T17:13:00.000Z 1564213 1.77
2019-07-06T17:11:13.000Z 1633680 4.27
2019-07-06T17:11:13.000Z 1560389 1.4
2019-07-06T17:08:11.000Z 1562300 1.4
2019-07-06T17:04:06.000Z 1060432 0.57
2019-07-06T16:59:51.000Z 1627409 1.47
2019-07-06T16:51:51.000Z 1409087 0.48
2019-07-06T16:39:52.000Z 1063560 0.51
2019-07-06T16:30:20.000Z 1297326 0.78
2019-07-06T16:26:12.000Z 917284 47.89
2019-07-06T16:17:43.000Z 1430960 2.99
2019-07-06T16:17:04.000Z 1634753 0.56
2019-07-06T16:16:41.000Z 1548877 5.34
2019-07-06T16:16:

2019-07-06T01:49:56.000Z 1500759 0.4825
2019-07-06T01:49:56.000Z 993569 0.4825
2019-07-06T01:49:56.000Z 1041167 0.4825
2019-07-06T01:49:56.000Z 1159630 0.4825
2019-07-06T01:49:56.000Z 1108004 0.4825
2019-07-06T01:49:56.000Z 822161 0.4825
2019-07-06T01:49:56.000Z 1338839 0.4825
2019-07-06T01:48:53.000Z 1638918 0.485
2019-07-06T01:48:53.000Z 1258770 0.485
2019-07-06T01:48:15.000Z 1150570 0.49
2019-07-06T01:35:41.000Z 1165664 0.488
2019-07-06T01:35:41.000Z 1389468 0.488
2019-07-06T01:35:41.000Z 982587 0.488
2019-07-06T01:35:41.000Z 1145039 0.488
2019-07-06T01:35:41.000Z 993579 0.488
2019-07-06T01:34:10.000Z 1487457 0.49
2019-07-06T01:34:10.000Z 1488404 0.49
2019-07-06T01:34:10.000Z 1266726 0.49
2019-07-06T01:34:10.000Z 1256310 0.49
2019-07-06T01:34:10.000Z 1150574 0.49
2019-07-06T01:26:09.000Z 965763 67.71
2019-07-06T01:23:51.000Z 1028480 2.64
2019-07-06T01:18:40.000Z 1624614 0.88
2019-07-06T00:27:26.000Z 907071 0.88
2019-07-06T00:26:11.000Z 917284 42.28
2019-07-06T00:24:16.000Z 1032632 2

2019-07-05T06:05:43.000Z 1637213 1.44
2019-07-05T06:04:33.000Z 1266780 0.46
2019-07-05T05:58:38.000Z 1637904 1.44
2019-07-05T05:58:18.000Z 1125884 1.3
2019-07-05T05:57:11.000Z 1637805 1.44
2019-07-05T05:49:58.000Z 1638427 1.44
2019-07-05T05:35:03.000Z 850434 3.74
2019-07-05T05:26:31.000Z 1010032 26.19
2019-07-05T04:56:14.000Z 1043670 38.21
2019-07-05T04:48:53.000Z 918525 2.13
2019-07-05T04:44:30.000Z 1636408 7.81
2019-07-05T04:32:34.000Z 1635424 0.54
2019-07-05T04:27:20.000Z 1637859 1.43
2019-07-05T04:26:06.000Z 857510 5.38
2019-07-05T04:20:16.000Z 1619303 86
2019-07-05T04:20:16.000Z 1613400 71.67
2019-07-05T03:57:42.000Z 1545051 37.2
2019-07-05T03:56:08.000Z 1016910 18.33
2019-07-05T03:26:10.000Z 853115 49.56
2019-07-05T03:11:15.000Z 1637404 16.55
2019-07-05T03:06:27.000Z 1625939 2.87
2019-07-05T02:55:47.000Z 1009482 72.15
2019-07-05T02:43:48.000Z 982615 0.54
2019-07-05T02:43:01.000Z 1586576 0.55
2019-07-05T02:43:01.000Z 1032794 0.57
2019-07-05T02:43:01.000Z 1634951 0.57
2019-07-05T02

2019-07-04T20:05:15.000Z 1606346 1.47
2019-07-04T20:05:15.000Z 1603754 1.47
2019-07-04T20:05:15.000Z 1603482 1.47
2019-07-04T20:04:57.000Z 1600062 1.47
2019-07-04T20:04:31.000Z 1606575 1.48
2019-07-04T20:04:31.000Z 1606993 1.48
2019-07-04T20:04:31.000Z 1601766 1.48
2019-07-04T19:56:08.000Z 1016910 18.87
2019-07-04T19:50:25.000Z 645929 1.15
2019-07-04T19:43:37.000Z 1621320 6.16
2019-07-04T19:43:05.000Z 1263411 0.49
2019-07-04T19:38:14.000Z 1632461 2.66
2019-07-04T19:31:45.000Z 1636453 147.56
2019-07-04T19:31:45.000Z 570885 498.28
2019-07-04T19:29:15.000Z 1636496 20.2
2019-07-04T19:26:00.000Z 853115 56.64
2019-07-04T19:26:00.000Z 1625044 1.18
2019-07-04T19:23:52.000Z 704920 1.18
2019-07-04T19:21:11.000Z 831242 0.88
2019-07-04T19:11:57.000Z 1588979 0.4
2019-07-04T19:04:20.000Z 1049303 12.69
2019-07-04T19:02:47.000Z 1261134 0.43
2019-07-04T19:02:17.000Z 1362199 0.4
2019-07-04T18:58:17.000Z 1432323 0.4
2019-07-04T18:58:17.000Z 993658 0.38
2019-07-04T18:57:45.000Z 1312819 0.37
2019-07-04T18:

...

2019-07-03T15:48:48.000Z 1541638 0.37
2019-07-03T15:47:39.000Z 1638053 2.91
2019-07-03T15:43:27.000Z 1637542 3.43
2019-07-03T15:29:32.000Z 1510864 2.07
2019-07-03T15:26:53.000Z 1037945 14.48
2019-07-03T15:25:29.000Z 1498793 2.07
2019-07-03T15:18:44.000Z 1636463 3.24
2019-07-03T15:17:55.000Z 1141457 0.38
2019-07-03T15:14:44.000Z 1363859 2.08
2019-07-03T15:10:50.000Z 1313210 2.7
2019-07-03T15:09:51.000Z 1363853 2.07
2019-07-03T15:08:29.000Z 993692 0.36
2019-07-03T15:07:07.000Z 1245534 0.36
2019-07-03T15:06:35.000Z 605244 0.35
2019-07-03T15:03:59.000Z 1363612 2.08
2019-07-03T15:03:59.000Z 1351314 2.08
2019-07-03T15:01:00.000Z 1624507 1.93
2019-07-03T14:56:09.000Z 1067790 46.06
2019-07-03T14:53:25.000Z 1339672 0.36
2019-07-03T14:53:13.000Z 1092759 0.36
2019-07-03T14:52:42.000Z 1433948 0.37
2019-07-03T14:52:29.000Z 563554 0.35
2019-07-03T14:48:06.000Z 581783 0.35
2019-07-03T14:48:06.000Z 607936 0.35
2019-07-03T14:43:18.000Z 617345 0.35
2019-07-03T14:42:47.000Z 594841 0.35
2019-07-03T14:39:4

2019-07-02T18:32:16.000Z 1630474 3.76
2019-07-02T18:26:48.000Z 995907 13.45
2019-07-02T18:22:03.000Z 622980 5.82
2019-07-02T18:14:48.000Z 637339 7.83
2019-07-02T17:59:11.000Z 1598364 0.37
2019-07-02T17:58:34.000Z 966604 35.28
2019-07-02T17:47:56.000Z 1611398 3.77
2019-07-02T17:35:52.000Z 1615438 0.37
2019-07-02T17:26:38.000Z 965763 67.14
2019-07-02T17:13:28.000Z 1114808 0.37
2019-07-02T17:12:53.000Z 1290830 5.22
2019-07-02T17:05:17.000Z 1558829 0.62
2019-07-02T16:58:02.000Z 1567875 0.4
2019-07-02T16:56:41.000Z 945435 27.29
2019-07-02T16:35:44.000Z 931546 1.44
2019-07-02T16:35:44.000Z 935468 1.44
2019-07-02T16:34:42.000Z 930614 1.44
2019-07-02T16:34:42.000Z 1541628 0.36
2019-07-02T16:30:36.000Z 1630751 15.93
2019-07-02T16:30:27.000Z 617546 5.76
2019-07-02T16:26:03.000Z 1636280 9.81
2019-07-02T16:15:53.000Z 498007 3.76
2019-07-02T16:12:24.000Z 1485999 1.47
2019-07-02T15:56:13.000Z 882455 23.4
2019-07-02T15:52:35.000Z 1615527 0.37
2019-07-02T15:52:35.000Z 1635868 3.47
2019-07-02T15:52:03.

2019-07-01T19:45:55.000Z 1626134 2.5
2019-07-01T19:42:32.000Z 565838 0.42
2019-07-01T19:35:32.000Z 1636496 16.42
2019-07-01T19:26:13.000Z 853115 54.68
2019-07-01T19:22:45.000Z 1636515 56.81
2019-07-01T19:15:58.000Z 1520774 0.84
2019-07-01T19:15:49.000Z 1623700 0.57
2019-07-01T19:07:46.000Z 1142341 0.45
2019-07-01T19:07:46.000Z 1116406 0.45
2019-07-01T19:06:44.000Z 1125828 1.3
2019-07-01T18:58:03.000Z 1637404 20.2
2019-07-01T18:57:41.000Z 1635981 2.57
2019-07-01T18:56:16.000Z 1009482 60.49
2019-07-01T18:27:30.000Z 1428441 0.42
2019-07-01T18:26:58.000Z 1063134 0.42
2019-07-01T18:26:23.000Z 1637229 1.42
2019-07-01T18:25:59.000Z 995907 15.79
2019-07-01T18:25:59.000Z 1637236 1.42
2019-07-01T18:25:54.000Z 1359391 0.52
2019-07-01T18:25:24.000Z 832125 43.05
2019-07-01T18:19:38.000Z 1637419 1.42
2019-07-01T18:19:38.000Z 643928 52.36
2019-07-01T18:19:14.000Z 1168834 48.2
2019-07-01T18:18:39.000Z 787965 42.53
2019-07-01T18:17:36.000Z 1046568 2.84
2019-07-01T17:58:15.000Z 966604 34.17
2019-07-01T1

2019-06-30T19:31:25.000Z 1625589 3.01
2019-06-30T19:20:14.000Z 1601896 1.13
2019-06-30T19:18:35.000Z 771163 0.46
2019-06-30T19:18:25.000Z 1097792 0.46
2019-06-30T19:17:22.000Z 993740 0.46
2019-06-30T19:17:22.000Z 1589899 0.45
2019-06-30T19:16:46.000Z 1211323 2.69
2019-06-30T19:16:46.000Z 1616037 2.99
2019-06-30T19:09:37.000Z 1628853 0.5
2019-06-30T19:09:37.000Z 1628846 0.5
2019-06-30T19:09:37.000Z 1618533 0.5
2019-06-30T19:09:37.000Z 1628863 0.5
2019-06-30T19:09:37.000Z 1618528 0.5
2019-06-30T19:09:37.000Z 1628873 0.5
2019-06-30T19:09:37.000Z 1626152 0.5
2019-06-30T19:09:37.000Z 1624399 0.5
2019-06-30T19:09:37.000Z 1043001 0.49
2019-06-30T19:09:37.000Z 1628851 0.5
2019-06-30T19:09:37.000Z 1119904 0.49
2019-06-30T19:09:37.000Z 1618725 0.5
2019-06-30T19:09:37.000Z 1063180 0.49
2019-06-30T19:09:37.000Z 1261045 0.49
2019-06-30T19:09:37.000Z 1628842 0.5
2019-06-30T19:09:37.000Z 1628870 0.5
2019-06-30T19:09:37.000Z 1428442 0.49
2019-06-30T19:06:15.000Z 845965 131.95
2019-06-30T19:05:43.000Z 

2019-06-30T10:37:47.000Z 1319297 5.28
2019-06-30T10:34:31.000Z 637541 5.84
2019-06-30T10:03:50.000Z 1120270 0.33
2019-06-30T09:32:54.000Z 1129917 4.04
2019-06-30T09:30:21.000Z 1261702 0.37
2019-06-30T09:27:41.000Z 1146686 2.48
2019-06-30T09:26:42.000Z 660506 1.02
2019-06-30T09:19:53.000Z 1144401 1.51
2019-06-30T09:18:52.000Z 1602988 1.54
2019-06-30T09:15:28.000Z 1407083 3.08
2019-06-30T09:08:56.000Z 627760 1.15
2019-06-30T09:04:27.000Z 1624639 2.48
2019-06-30T09:01:42.000Z 1093691 0.36
2019-06-30T08:47:54.000Z 1360988 15.81
2019-06-30T08:44:32.000Z 1126415 2.48
2019-06-30T08:44:32.000Z 636714 1.14
2019-06-30T08:44:32.000Z 646645 1.04
2019-06-30T08:42:43.000Z 612509 1.14
2019-06-30T08:42:26.000Z 1581468 0.4
2019-06-30T08:42:26.000Z 571483 1.28
2019-06-30T08:42:26.000Z 1540078 1.45
2019-06-30T08:42:13.000Z 1160510 1.47
2019-06-30T08:41:54.000Z 1168044 1.48
2019-06-30T08:40:57.000Z 1063241 0.37
2019-06-30T08:19:03.000Z 1433095 0.36
2019-06-30T08:17:43.000Z 993778 0.35
2019-06-30T08:11:32.

In [40]:
new_dense = pd.DataFrame(results['results'], columns=['time', 'ID', 'USD'])
new_dense.to_csv("new_dense_DELETE.csv", index = False)

In [41]:
data = pd.read_csv("new_dense_DELETE.csv")
data.head(5)

,time,ID,USD
0,2019-07-07T06:37:59.000Z,1561681,1.10
1,2019-07-07T06:32:17.000Z,1553924,0.32
2,2019-07-07T06:31:26.000Z,1555773,0.32
3,2019-07-07T06:29:29.000Z,1554209,0.34
4,2019-07-07T06:26:32.000Z,1552162,0.33


## TIME TO PULL FROM API

### Create new columns

In [42]:
# Random
data['generation'] = ''
data['color'] = ''
data['is_fancy'] = ''
data['is_exclusive'] = ''
data['fancy_type'] = ''
data['is_prestige'] = ''
data['prestige_type'] = ''
data['prestige_ranking'] = ''
data['fancy_ranking'] = ''
data['prestige_time_limit'] = ''

# Enhanced cattributes
data['enhanced_cattributes_eyes'] = '' 
data['enhanced_cattributes_pattern'] = '' 
data['enhanced_cattributes_body'] = '' 
data['enhanced_cattributes_mouth'] = '' 
data['enhanced_cattributes_colorprimary'] = '' 
data['enhanced_cattributes_colorsecondary'] = '' 
data['enhanced_cattributes_colortertiary'] = '' 
data['enhanced_cattributes_coloreyes'] = '' 

# Status
data['status_cooldown_index'] = ''
data['status_is_ready'] = ''
data['status_is_gestating'] = ''

# Purrs
data['purrs_count'] = ''
data['purrs_is_purred'] = ''

# watchlist
data['watchlist_count'] = ''
data['watchlist_is_watchlisted'] = ''


url_list = ['https://public.api.cryptokitties.co/v1/kitties/' + str(i) for i in data['ID'].unique()]

print(len(url_list))

2187


## DELETE PREVIOUS HTML FILES

In [48]:
# import shutil
# shutil.rmtree('html/')
os.mkdir("html")

In [49]:
import aiofiles
import os
import asyncio
import nest_asyncio
nest_asyncio.apply()
import aiohttp


payload = {}
headers = {
  'x-api-token': 'w-5V-v6V__trBl7yxGMXnwhLiGkq-03XnCVNaKePPd4',
  'Content-Type': 'application/json',
}

async def download_html(session, url):
    async with session.request('GET', url, headers = headers, data = payload, allow_redirects=False) as res:
        filename = f'html/output_{os.path.basename(url)}.html'
        print(len(os.listdir("html/")), end = "\r")
        async with aiofiles.open(filename, 'wb') as f:
            while True:
                chunk = await res.content.read(1024)
                if not chunk:
                    break
                await f.write(chunk)

        return await res.release()
    
async def main(url):
    connector = aiohttp.TCPConnector(limit=60)
    async with aiohttp.ClientSession(connector=connector) as session: #aiohttp.ClientSession()
        await download_html(session, url)

# urls = ['https://public.api.cryptokitties.co/v1/kitties/' + str(i) for i in range(100)] # a big list of websites to scrape

for i in range(int(len(url_list)/100)+1):
    urls = url_list[i*100:i*100+100]
    print('step: ' + str(i) + '   Number of files: ' + str(len(os.listdir("./html"))))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(
        asyncio.gather(*(main(url) for url in urls))
    )

step: 0   Number of files: 0
step: 1   Number of files: 100
step: 2   Number of files: 200
step: 3   Number of files: 300
step: 4   Number of files: 400
step: 5   Number of files: 500
step: 6   Number of files: 600
step: 7   Number of files: 700
step: 8   Number of files: 800
step: 9   Number of files: 900
step: 10   Number of files: 1000
step: 11   Number of files: 1100
step: 12   Number of files: 1200
step: 13   Number of files: 1300
step: 14   Number of files: 1400
step: 15   Number of files: 1500
step: 16   Number of files: 1600
step: 17   Number of files: 1700
step: 18   Number of files: 1800
step: 19   Number of files: 1900
step: 20   Number of files: 2000
step: 21   Number of files: 2100


## MIGHT HAVE TO DEAL WITH ERROR HTML FILES AT SOME POINT!

In [58]:
def data_to_csv(data):
    for k in range(len(data)):
        i_d = data.iloc[k, 1]

        file= codecs.open("html/output_" + str(i_d) + ".html", 'r',  encoding="utf8")
        parsed = json.loads(file.read())

        # Random
        data.iloc[k, 3] = parsed['generation']        
        data.iloc[k, 4] = parsed['color']
        data.iloc[k, 5] = parsed['is_fancy']
        data.iloc[k, 6] = parsed['is_exclusive']
        data.iloc[k, 7] = parsed['fancy_type']
        data.iloc[k, 8] = parsed['is_prestige']
        data.iloc[k, 9] = parsed['prestige_type']
        data.iloc[k, 10] = parsed['prestige_ranking']
        data.iloc[k, 11] = parsed['fancy_ranking']
        data.iloc[k, 12] = parsed['prestige_time_limit']

        # Enhanced cattributes
        if parsed['enhanced_cattributes'] != []:
            data.iloc[k, 13] = parsed['enhanced_cattributes'][0]['description']
            data.iloc[k, 14] = parsed['enhanced_cattributes'][1]['description']
            data.iloc[k, 15] = parsed['enhanced_cattributes'][2]['description']
            data.iloc[k, 16] = parsed['enhanced_cattributes'][3]['description']
            data.iloc[k, 17] = parsed['enhanced_cattributes'][4]['description']
            data.iloc[k, 18] = parsed['enhanced_cattributes'][5]['description']
            data.iloc[k, 19] = parsed['enhanced_cattributes'][6]['description'] 
            data.iloc[k, 20] = parsed['enhanced_cattributes'][7]['description']

        # Status
        data.iloc[k, 21] = parsed['status']['cooldown_index']
        data.iloc[k, 22] = parsed['status']['is_ready']
        data.iloc[k, 23] = parsed['status']['is_gestating']

        # Purrs
        data.iloc[k, 24] = parsed['purrs']['count']
        data.iloc[k, 25] = parsed['purrs']['is_purred']

        # watchlist
        data.iloc[k, 26] = parsed['watchlist']['count']
        data.iloc[k, 27] = parsed['watchlist']['is_watchlisted']


        print(f"we at: {k} out of {len(data)}. Giving us percent complete of {round(100*k/len(data), 5)}%   ", end = "\r")
        
        if k % 100 == 0:
            data.to_csv("new_dense_DELETE.csv", index = False)
    return data

In [59]:
d = data_to_csv(data)

In [69]:
new_data = d
new_data.head()

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,2019-07-07T06:37:59.000Z,1561681,1.10,5,eclipse,False,False,None,False,None,None,None,None,selkirk,splat,wiley,ducky,fangtastic,greymatter,royalpurple,frosting,3,True,False,0,False,0,False
1,2019-07-07T06:32:17.000Z,1553924,0.32,8,doridnudibranch,False,False,None,False,None,None,None,None,spock,ragdoll,baddate,salty,rollercoaster,ducky,greymatter,cerulian,9,True,False,0,False,0,False
2,2019-07-07T06:31:26.000Z,1555773,0.32,7,coralsunrise,False,False,None,False,None,None,None,None,thicccbrowz,moue,tiger,salty,highlander,ducky,salmon,swampgreen,7,True,False,0,False,0,False
3,2019-07-07T06:29:29.000Z,1554209,0.34,6,sapphire,False,False,None,False,None,None,None,None,totesbasic,savannah,moue,baddate,salty,ducky,salmon,coffee,9,True,False,0,False,0,False
4,2019-07-07T06:26:32.000Z,1552162,0.33,8,sapphire,False,False,None,False,None,None,None,None,thicccbrowz,grim,salty,koladiviya,rorschach,ducky,cottoncandy,swampgreen,12,False,False,0,False,0,False


In [63]:
old_data = pd.read_csv("train_complete.csv")
old_data.head()

,time,ID,USD,generation,color,is_fancy,is_exclusive,fancy_type,is_prestige,prestige_type,prestige_ranking,fancy_ranking,prestige_time_limit,enhanced_cattributes_eyes,enhanced_cattributes_pattern,enhanced_cattributes_body,enhanced_cattributes_mouth,enhanced_cattributes_colorprimary,enhanced_cattributes_colorsecondary,enhanced_cattributes_colortertiary,enhanced_cattributes_coloreyes,status_cooldown_index,status_is_ready,status_is_gestating,purrs_count,purrs_is_purred,watchlist_count,watchlist_is_watchlisted
0,2019-06-30T02:21:40.000Z,1632691,0.96,11.0,thundergrey,False,False,NaN,False,NaN,NaN,NaN,NaN,confuzzled,serpent,birman,moonrise,dragontail,frozen,greymatter,chocolate,5.0,True,False,1.0,False,0.0,False
1,2019-06-30T02:16:25.000Z,1093660,0.40,2.0,mintgreen,False,False,NaN,False,NaN,NaN,NaN,NaN,calicool,chronic,majestic,highlander,nachocheez,egyptiankohl,purplehaze,mintgreen,1.0,True,False,1.0,False,0.0,False
2,2019-06-30T02:05:20.000Z,1063243,0.38,4.0,cyan,False,False,NaN,False,NaN,NaN,NaN,NaN,sphynx,soserious,swarley,rorschach,aflutter,shadowgrey,padparadscha,belleblue,2.0,True,False,1.0,False,0.0,False
3,2019-06-30T02:04:43.000Z,1043057,0.39,2.0,doridnudibranch,False,False,NaN,False,NaN,NaN,NaN,NaN,soserious,birman,swarley,rorschach,cinderella,royalpurple,purplehaze,doridnudibranch,1.0,True,False,0.0,False,0.0,False
4,2019-06-30T01:45:54.000Z,1634788,14.65,14.0,cyan,True,False,Magmeow,False,NaN,NaN,509.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,True,False,2.0,False,0.0,False


In [65]:
old_data.shape

(106261, 28)

In [67]:
new_data.shape

(2441, 28)

In [72]:
new_data = new_data.append(old_data)

In [74]:
new_data.to_csv("train_complete.csv", index = False)

In [78]:
pd.read_csv("train_complete.csv").shape

(108702, 28)